In [1]:
"""
Step 0: Import Libraries and download necessary NLTK resources
"""
import os
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, LatentDirichletAllocation, TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
# As my Python Interpreter does not support PyMuPDF, I will use pdfplumber instead
import pdfplumber  # Using pdfplumber instead of PyMuPDF
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.corpora import Dictionary
from gensim.models import LsiModel
import warnings
import argparse

warnings.filterwarnings('ignore')

# Download necessary NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

/Users/yiningxiang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

In [2]:
"""
Step 1: Define the PDFProcessor class to handle PDF files
        We use pdfplumber to extract text from PDF files.
"""
class PDFProcessor:
    def __init__(self, directory_path):
        self.directory_path = directory_path
        self.pdf_files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]
        self.documents = {}

    def extract_all_texts(self):
        for pdf_file in self.pdf_files:
            file_path = os.path.join(self.directory_path, pdf_file)
            self.documents[pdf_file] = self.extract_text(file_path)
        return self.documents

    @staticmethod
    def extract_text(file_path):
        try:
            text = ""
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    extracted = page.extract_text()
                    if extracted:
                        text += extracted + "\n"
            return text
        except Exception as e:
            print(f"Error extracting text from {file_path}: {e}")
            return ""

In [3]:
"""
Step 2: Define the TextPreprocessor class to handle text preprocessing
"""
class TextPreprocessor:
    def __init__(self, additional_stopwords=None):
        self.stop_words = set(stopwords.words('english'))
        if additional_stopwords:
            self.stop_words.update(additional_stopwords)
        self.lemmatizer = WordNetLemmatizer()

    def preprocess(self, text):
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        tokens = word_tokenize(text)
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens if
                  token not in self.stop_words and len(token) > 2]
        return ' '.join(tokens)

    def preprocess_documents(self, documents):
        return {doc_name: self.preprocess(text) for doc_name, text in documents.items()}


In [4]:
"""
Step 3: Define the DocumentClustering class to handle document clustering
"""
class DocumentClustering:
    def __init__(self, n_clusters=2):
        self.n_clusters = n_clusters
        self.vectorizer = TfidfVectorizer(max_features=5000, min_df=2, max_df=0.85)
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        self.document_names = None
        self.tfidf_matrix = None
        self.pca = PCA(n_components=2, random_state=42)
        self.labels = None

    def fit(self, preprocessed_documents):
        self.document_names = list(preprocessed_documents.keys())
        texts = [preprocessed_documents[doc] for doc in self.document_names]
        self.tfidf_matrix = self.vectorizer.fit_transform(texts)
        self.labels = self.kmeans.fit_predict(self.tfidf_matrix)
        return self

    def get_cluster_terms(self, n_terms=20):
        feature_names = self.vectorizer.get_feature_names_out()
        cluster_terms = {}
        order_centroids = self.kmeans.cluster_centers_.argsort()[:, ::-1]
        for cluster_id in range(self.n_clusters):
            top_term_indices = order_centroids[cluster_id, :n_terms]
            top_terms = [feature_names[i] for i in top_term_indices]
            cluster_terms[cluster_id] = top_terms
        return cluster_terms

    def identify_outliers(self, percentile=95):
        distances = np.min(
            [np.linalg.norm(self.tfidf_matrix.toarray() - center, axis=1)
             for center in self.kmeans.cluster_centers_],
            axis=0
        )
        threshold = np.percentile(distances, percentile)
        outlier_indices = np.where(distances > threshold)[0]
        outlier_indices = list(set(outlier_indices))
        outliers = [self.document_names[i] for i in outlier_indices]
        return outliers


    def get_cluster_documents(self):
        cluster_docs = {}
        for cluster_id in range(self.n_clusters):
            indices = np.where(self.labels == cluster_id)[0]
            cluster_docs[cluster_id] = [self.document_names[i] for i in indices]
        return cluster_docs

In [5]:
"""
Classifier 1
Based on the clustering results, we can create a similarity classifier.
We can use KMeans clustering to create a similarity classifier similar to unsupervised learning.
"""
class SimilarityClassifier:
    def __init__(self, vectorizer, kmeans_model, cluster_mapping=None):
        self.vectorizer = vectorizer
        self.kmeans_model = kmeans_model
        self.cluster_mapping = cluster_mapping or {}
        self.threshold = 0.2

    def classify(self, text, preprocess_func=None):
        if preprocess_func:
            text = preprocess_func(text)
        text_tfidf = self.vectorizer.transform([text])
        distances = [np.linalg.norm(text_tfidf.toarray() - center)
                     for center in self.kmeans_model.cluster_centers_]
        closest_cluster = np.argmin(distances)
        min_distance = distances[closest_cluster]
        is_outlier = min_distance > self.threshold
        predicted_class = self.cluster_mapping.get(closest_cluster, f"Cluster {closest_cluster}")
        max_distance = max(distances)
        confidence = 1 - (min_distance / max_distance) if max_distance > 0 else 1.0
        return predicted_class, confidence, is_outlier


In [6]:
"""
Classifier 2
We first manually read some files and create a list of keywords for each category.
Then we repeatedly backtest and revise the keywords based on the training results.
"""
class KeywordClassifier:
    def __init__(self):
        self.accounting_keywords = [
            'accounting', 'principles', 'standards', 'financial reporting', 'accounting standards',
            'financial statement', 'balance sheet', 'income statement', 'cash flow',
            'accounting', 'audit', 'ledger', 'journal entry', 'debit', 'credit',
            'accounts payable', 'accounts receivable', 'asset', 'liability', 'equity',
            'taxation', 'financial reporting', 'bookkeeping', 'accrual', 'depreciation',
            'amortization', 'inventory', 'cost accounting', 'budgeting', 'variance analysis',
            'profit', 'loss', 'revenue recognition', 'internal control', 'ifrs', 'gaap'
        ]
        self.finance_keywords = [
            'investment', 'portfolio', 'risk', 'return', 'capital', 'valuation',
            'interest rate', 'bond', 'stock', 'market', 'security', 'option', 'futures',
            'derivative', 'dividend', 'corporate finance', 'capm', 'present value',
            'npv', 'irr', 'wacc', 'capital structure', 'leverage', 'beta', 'alpha',
            'financial market', 'efficient market', 'arbitrage', 'hedging', 'diversification',
            'financial management', 'merger', 'acquisition'
        ]
        self.syllabus_keywords = [
            'syllabus', 'course outline', 'learning objective', 'prerequisite',
            'textbook', 'required reading', 'grading', 'assessment', 'assignment',
            'lecture', 'class schedule', 'course description', 'instructor', 'professor',
            'office hours', 'academic integrity', 'plagiarism', 'course policy',
            'attendance', 'participation', 'final exam', 'midterm'
        ]

    def is_syllabus(self, text):
        if text is None:
            return False
        text_lower = text.lower()
        syllabus_count = sum(text_lower.count(keyword) for keyword in self.syllabus_keywords)
        return syllabus_count >= 5

    def classify_by_filename(self, filename):
        if filename is None:
            return "Unknown", 0.0
        filename_lower = filename.lower()
        accounting_indicators = ['acc', 'acct', 'accounting']
        has_accounting = any(indicator in filename_lower for indicator in accounting_indicators)
        finance_indicators = ['fin', 'finance']
        has_finance = any(indicator in filename_lower for indicator in finance_indicators)
        if has_accounting:
            return "Accounting", 0.9
        elif has_finance:
            return "Finance", 0.3
        else:
            return "Unknown", 0.0

    def classify(self, text, filename=None):
        if text is None:
            return "Unknown", 0.0
        text = text.lower()
        accounting_count = sum(text.count(keyword) for keyword in self.accounting_keywords)
        finance_count = sum(text.count(keyword) for keyword in self.finance_keywords)
        total_count = accounting_count + finance_count
        if total_count == 0:
            keyword_class = "Unknown"
            keyword_confidence = 0.0
        elif accounting_count >= finance_count:
            keyword_confidence = accounting_count / (total_count or 1)
            keyword_class = "Accounting"
        else:
            keyword_confidence = finance_count / (total_count or 1)
            keyword_class = "Finance"
        if filename is None:
            return keyword_class, keyword_confidence
        filename_class, filename_confidence = self.classify_by_filename(filename)
        if filename_class == keyword_class and filename_class != "Unknown":
            confidence = (0.7 * keyword_confidence) + (0.3 * filename_confidence)
            return keyword_class, min(confidence, 0.95)
        elif filename_class != "Unknown":
            confidence = (0.8 * keyword_confidence) + (0.2 * filename_confidence)
            return keyword_class, confidence
        else:
            return keyword_class, keyword_confidence


In [7]:
"""
Classifier 3
We use a more advanced classifier by comparing the similarity of all document with the reference documents.
Instead of manually setting the reference document, we ask the computer to find two documents, one for finance and one for accounting, that best represent their respective category.
( seen in `set_reference_document` function under SyllabiAnalyzer )
This is similar to supervised learning, but we do not need to manually label the documents nor select the reference documents.
"""
class DocumentSimilarityClassifier:
    def __init__(self, reference_accounting_doc=None, reference_finance_doc=None):
        self.reference_accounting_doc = reference_accounting_doc
        self.reference_finance_doc = reference_finance_doc
        self.preprocessor = TextPreprocessor()

    def calculate_similarity(self, text, reference_doc):
        if not reference_doc:
            return 0.0
        preprocessed_text = self.preprocessor.preprocess(text)
        preprocessed_reference = self.preprocessor.preprocess(reference_doc)
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform([preprocessed_text, preprocessed_reference])
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        return similarity

    def classify(self, text):
        accounting_similarity = self.calculate_similarity(text, self.reference_accounting_doc)
        finance_similarity = self.calculate_similarity(text, self.reference_finance_doc)
        if accounting_similarity > finance_similarity:
            predicted_class = "Accounting"
            confidence = accounting_similarity
        else:
            predicted_class = "Finance"
            confidence = finance_similarity
        return predicted_class, confidence, (accounting_similarity, finance_similarity)


In [8]:
"""
Classifier 3.5
This classifier is strictly speaking not a new classifier, it is just a combination of the KeywordClassifier and DocumentSimilarityClassifier.
It calculates the weighted average of the keyword and similarity scores to determine the final classification."""
class CombinedClassifier:
    def __init__(self, keyword_classifier, similarity_classifier, accounting_weight=0.5, finance_weight=0.5):
        self.keyword_classifier = keyword_classifier
        self.similarity_classifier = similarity_classifier
        self.accounting_weight = accounting_weight
        self.finance_weight = finance_weight
        self.outlier_threshold = 0.4

    def classify(self, text, filename=None):
        is_syllabus = self.keyword_classifier.is_syllabus(text)
        if not is_syllabus:
            return "Not a Syllabus", 0.0, True, {}
        keyword_class, keyword_conf = self.keyword_classifier.classify(text, filename)
        similarity_class, similarity_conf, (
        accounting_similarity, finance_similarity) = self.similarity_classifier.classify(text)

        accounting_score = 0.0
        finance_score = 0.0
        if keyword_class == "Accounting":
            accounting_score = (self.accounting_weight * keyword_conf) + (
                        (1 - self.accounting_weight) * accounting_similarity)
        else:
            accounting_score = accounting_similarity
        if keyword_class == "Finance":
            finance_score = (self.finance_weight * keyword_conf) + ((1 - self.finance_weight) * finance_similarity)
        else:
            finance_score = finance_similarity

        is_outlier = False
        if max(accounting_score, finance_score) < self.outlier_threshold:
            predicted_class = "Potential Outlier"
            confidence = max(accounting_score, finance_score)
            is_outlier = True
        elif accounting_score > finance_score:
            predicted_class = "Accounting"
            confidence = accounting_score
        else:
            predicted_class = "Finance"
            confidence = finance_score

        details = {
            "keyword_classification": (keyword_class, keyword_conf),
            "similarity_classification": (similarity_class, similarity_conf),
            "accounting_similarity": accounting_similarity,
            "finance_similarity": finance_similarity,
            "final_accounting_score": accounting_score,
            "final_finance_score": finance_score
        }

        return predicted_class, confidence, is_outlier, details


In [9]:
"""
Classifier 4
We use LDA and LSI to classify the documents.
TopicModeler differs from the other classifiers by focusing on underlying thematic patterns rather than direct feature comparison. Unlike KeywordClassifier (which matches predefined terms) or SimilarityClassifier (which compares vector distances), TopicModeler discovers latent semantic structures in the text using probabilistic models (LDA) and matrix factorization (LSI). It can reveal conceptual relationships between documents that might not be obvious through direct keyword matching or similarity calculations.
"""
class TopicModeler:
    def __init__(self, n_topics=2):
        self.n_topics = n_topics
        self.lda_model = LatentDirichletAllocation(
            n_components=n_topics,
            random_state=42,
            max_iter=10,
            learning_method='online'
        )
        self.lsi_model = None
        self.dictionary = None
        self.corpus = None
        self.vectorizer = TfidfVectorizer(max_features=5000, min_df=2, max_df=0.85)

    def fit_lda(self, preprocessed_documents):
        texts = list(preprocessed_documents.values())
        self.tfidf_matrix = self.vectorizer.fit_transform(texts)
        self.lda_model.fit(self.tfidf_matrix)
        return self

    def fit_lsi(self, preprocessed_documents):
        texts = list(preprocessed_documents.values())
        tokenized_texts = [text.split() for text in texts]
        self.dictionary = Dictionary(tokenized_texts)
        self.corpus = [self.dictionary.doc2bow(text) for text in tokenized_texts]
        self.lsi_model = LsiModel(
            self.corpus,
            id2word=self.dictionary,
            num_topics=self.n_topics
        )
        return self

    def get_lda_topics(self, n_terms=10):
        feature_names = self.vectorizer.get_feature_names_out()
        topics = []
        for topic_idx, topic in enumerate(self.lda_model.components_):
            top_terms_idx = topic.argsort()[:-n_terms - 1:-1]
            top_terms = [feature_names[i] for i in top_terms_idx]
            topics.append(top_terms)
        return topics

    def get_lsi_topics(self, n_terms=10):
        topics = []
        for topic_id in range(self.n_topics):
            top_terms = self.lsi_model.show_topic(topic_id, n_terms)
            topics.append([term for term, _ in top_terms])
        return topics

    def classify_with_lda(self, text, preprocess_func=None):
        if preprocess_func:
            text = preprocess_func(text)
        text_tfidf = self.vectorizer.transform([text])
        topic_dist = self.lda_model.transform(text_tfidf)[0]
        return topic_dist.tolist()

    def classify_with_lsi(self, text, preprocess_func=None):
        if preprocess_func:
            text = preprocess_func(text)
        tokenized_text = text.split()
        bow = self.dictionary.doc2bow(tokenized_text)
        topic_dist = self.lsi_model[bow]
        dense_vec = np.zeros(self.n_topics)
        for topic_id, weight in topic_dist:
            dense_vec[topic_id] = weight
        return dense_vec.tolist()


In [16]:

class SyllabiAnalyzer:
    def __init__(self, directory_path):
        self.directory_path = directory_path
        self.pdf_processor = PDFProcessor(directory_path)
        self.text_preprocessor = TextPreprocessor()
        self.clustering = DocumentClustering()
        self.raw_documents = {}
        self.preprocessed_documents = {}
        self.similarity_classifier = None
        self.keyword_classifier = KeywordClassifier()
        self.topic_modeler = TopicModeler()
        self.reference_accounting_doc = None
        self.reference_finance_doc = None
        self.document_similarity_classifier = None
        self.combined_classifier = None

    def set_reference_documents(self, accounting_filename, finance_filename):
        accounting_path = os.path.join(self.directory_path, accounting_filename)
        finance_path = os.path.join(self.directory_path, finance_filename)
        self.reference_accounting_doc = PDFProcessor.extract_text(accounting_path)
        self.reference_finance_doc = PDFProcessor.extract_text(finance_path)
        self.document_similarity_classifier = DocumentSimilarityClassifier(
            self.reference_accounting_doc,
            self.reference_finance_doc
        )
        self.combined_classifier = CombinedClassifier(
            self.keyword_classifier,
            self.document_similarity_classifier
        )
        return self

    def process_documents(self):
        print("Extracting text from PDF files...")
        self.raw_documents = self.pdf_processor.extract_all_texts()
        print(f"Extracted text from {len(self.raw_documents)} files.")
        print("Preprocessing documents...")
        self.preprocessed_documents = self.text_preprocessor.preprocess_documents(self.raw_documents)
        return self

    def perform_clustering(self):
        print("Performing document clustering...")
        self.clustering.fit(self.preprocessed_documents)
        self.similarity_classifier = SimilarityClassifier(
            self.clustering.vectorizer,
            self.clustering.kmeans
        )
        return self

    def analyze_clusters(self):
        cluster_terms = self.clustering.get_cluster_terms()
        cluster_docs = self.clustering.get_cluster_documents()
        outliers = self.clustering.identify_outliers(percentile=95)
        total_docs = sum(len(docs) for docs in cluster_docs.values()) + len(outliers)

        # Remove outliers from cluster_docs
        for cluster_id in cluster_docs:
            cluster_docs[cluster_id] = [doc for doc in cluster_docs[cluster_id] if doc not in outliers]

        total_docs = sum(len(docs) for docs in cluster_docs.values()) + len(outliers)

        print("\nCluster Analysis:")
        print("-----------------")
        print(f"Total documents: {len(self.raw_documents)}")
        print(f"Documents in clusters + outliers: {total_docs}")

        common_terms = {'student', 'class', 'exam', 'academic', 'assignment', 'chapter',
                        'grade', 'http', 'may', 'final', 'course', 'will',
                        'material', 'case', 'instructor', 'university', 'quiz', 'policy',
                        'please', 'lecture', 'week'}

        accounting_terms = {'accounting', 'balance', 'statement', 'ledger', 'debit', 'credit',
                            'taxation', 'ifrs', 'gaap', 'audit', 'bookkeeping', 'accrual',
                            'depreciation', 'asset', 'liability', 'equity', 'journal', 'income',
                            'cost', 'inventory', 'revenue', 'expense', 'tax', 'sheet', 'financial'}

        finance_terms = {'finance', 'investment', 'portfolio', 'risk', 'return', 'capital',
                         'valuation', 'market', 'security', 'bond', 'stock', 'derivative',
                         'dividend', 'leverage', 'beta', 'alpha', 'arbitrage', 'hedging',
                         'wealth', 'banking', 'interest', 'loan', 'mortgage', 'funding'}

        cluster_mapping = {}
        for cluster_id, terms in cluster_terms.items():
            if "accounting" in terms[:5]:
                cluster_label = "Accounting"
                cluster_mapping[cluster_id] = cluster_label

                display_terms = []
                term_index = 0
                while len(display_terms) < 10 and term_index < len(terms):
                    if terms[term_index] not in common_terms or len(display_terms) >= sum(
                            1 for t in terms if t not in common_terms):
                        display_terms.append(terms[term_index])
                    term_index += 1

                print(f"\nCluster {cluster_id} (likely {cluster_label} - 'accounting' found in top terms):")
                print(f"Number of documents: {len(cluster_docs[cluster_id])}")
                print(f"Top terms: {', '.join(display_terms)}")
                print(f"Sample documents: {', '.join(cluster_docs[cluster_id][:3])}")
                continue

            specific_terms = [term for term in terms if term not in common_terms]
            accounting_count = sum(1 for term in specific_terms[:20] if term in accounting_terms)
            finance_count = sum(1 for term in specific_terms[:20] if term in finance_terms)

            if accounting_count > finance_count:
                cluster_label = "Accounting"
            else:
                cluster_label = "Finance"

            cluster_mapping[cluster_id] = cluster_label

            display_terms = []
            term_index = 0
            while len(display_terms) < 10 and term_index < len(terms):
                if terms[term_index] not in common_terms or len(display_terms) >= len(specific_terms):
                    display_terms.append(terms[term_index])
                term_index += 1

            print(f"\nCluster {cluster_id} (likely {cluster_label}):")
            print(f"Number of documents: {len(cluster_docs[cluster_id])}")
            print(f"Top terms: {', '.join(display_terms)}")
            print(f"Accounting terms: {accounting_count}, Finance terms: {finance_count}")
            print(f"Sample documents: {', '.join(cluster_docs[cluster_id][:3])}")

        print("\nPotential Outliers:")
        print(f"Number of outliers: {len(outliers)}")
        if outliers:
            print(f"Outlier documents: {', '.join(outliers[:5])}")

        self.similarity_classifier.cluster_mapping = cluster_mapping
        return cluster_mapping


    def perform_topic_modeling(self):
        print("\nPerforming topic modeling...")
        self.topic_modeler.fit_lda(self.preprocessed_documents)
        self.topic_modeler.fit_lsi(self.preprocessed_documents)
        lda_topics = self.topic_modeler.get_lda_topics()
        lsi_topics = self.topic_modeler.get_lsi_topics()

        print("\nLDA Topics:")
        for i, topic in enumerate(lda_topics):
            print(f"Topic {i}: {', '.join(topic)}")

        print("\nLSI Topics:")
        for i, topic in enumerate(lsi_topics):
            print(f"Topic {i}: {', '.join(topic)}")

        return self

    def evaluate_classification_methods(self, test_documents=None):
        if test_documents is None:
            test_docs = {}
            for i, (name, text) in enumerate(self.raw_documents.items()):
                if i % 5 == 0:  # Use every 5th document as test
                    test_docs[name] = text
            test_documents = test_docs

        results = {}
        print("\nEvaluating Classification Methods:")
        print("---------------------------------")

        for doc_name, raw_text in test_documents.items():
            preprocessed_text = self.text_preprocessor.preprocess(raw_text)

            # Similarity-based classification
            sim_class, sim_conf, is_outlier = self.similarity_classifier.classify(
                raw_text,
                preprocess_func=self.text_preprocessor.preprocess
            )

            # Keyword-based classification
            kw_class, kw_conf = self.keyword_classifier.classify(raw_text)

            # LDA-based classification
            lda_dist = self.topic_modeler.classify_with_lda(
                raw_text,
                preprocess_func=self.text_preprocessor.preprocess
            )
            lda_class = "Accounting" if lda_dist[0] > lda_dist[1] else "Finance"
            lda_conf = max(lda_dist)

            # LSI-based classification
            lsi_dist = self.topic_modeler.classify_with_lsi(
                raw_text,
                preprocess_func=self.text_preprocessor.preprocess
            )
            lsi_class = "Accounting" if abs(lsi_dist[0]) > abs(lsi_dist[1]) else "Finance"
            lsi_conf = max(abs(val) for val in lsi_dist)

            # Combined classifier if available
            if self.combined_classifier:
                comb_class, comb_conf, comb_is_outlier, comb_details = self.combined_classifier.classify(
                    raw_text, doc_name
                )
                combined_result = (comb_class, comb_conf, comb_is_outlier)
            else:
                combined_result = None

            results[doc_name] = {
                "similarity": (sim_class, sim_conf, is_outlier),
                "keyword": (kw_class, kw_conf),
                "lda": (lda_class, lda_conf),
                "lsi": (lsi_class, lsi_conf),
                "combined": combined_result
            }

            print(f"\nDocument: {doc_name}")
            print(f"Similarity Classification: {sim_class} (Confidence: {sim_conf:.2f}, Outlier: {is_outlier})")
            print(f"Keyword Classification: {kw_class} (Confidence: {kw_conf:.2f})")
            print(f"LDA Classification: {lda_class} (Confidence: {lda_conf:.2f})")
            print(f"LSI Classification: {lsi_class} (Confidence: {lsi_conf:.2f})")
            if combined_result:
                print(
                    f"Combined Classification: {comb_class} (Confidence: {comb_conf:.2f}, Outlier: {comb_is_outlier})")

        return results

    def run_analysis(self):
        self.process_documents()
        self.perform_clustering()
        cluster_mapping = self.analyze_clusters()
        self.perform_topic_modeling()

        # Try to find reference documents for document similarity classifier
        accounting_docs = []
        finance_docs = []

        for cluster_id, label in cluster_mapping.items():
            docs = self.clustering.get_cluster_documents()[cluster_id]
            if label == "Accounting" and docs:
                accounting_docs = docs
            elif label == "Finance" and docs:
                finance_docs = docs

        # If we found some accounting and finance documents, set up the reference docs
        if accounting_docs and finance_docs:
            self.set_reference_documents(accounting_docs[0], finance_docs[0])
            print(f"\nSet up reference documents: {accounting_docs[0]} (Accounting) and {finance_docs[0]} (Finance)")

        evaluation_results = self.evaluate_classification_methods()

        return self


In [15]:
directory = 'Syllabi0_8'
file_to_classify = 'your_file.pdf'

analyzer = SyllabiAnalyzer(directory)
analyzer.process_documents()
analyzer.perform_clustering()
analyzer.analyze_clusters()

text = PDFProcessor.extract_text(file_to_classify)

sim_class, sim_conf, is_outlier = analyzer.similarity_classifier.classify(
    text,
    preprocess_func=analyzer.text_preprocessor.preprocess
)

kw_class, kw_conf = analyzer.keyword_classifier.classify(text)

print(f"\nClassification Results for {file_to_classify}:")
print(f"Similarity Classification: {sim_class} (Confidence: {sim_conf:.2f}, Outlier: {is_outlier})")
print(f"Keyword Classification: {kw_class} (Confidence: {kw_conf:.2f})")

Extracting text from PDF files...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Extracted text from 107 files.
Preprocessing documents...
Performing document clustering...

Cluster Analysis:
-----------------
Total documents: 107
Documents in clusters + outliers: 107

Cluster 0 (likely Finance):
Number of documents: 52
Top terms: finance, instructor, university, quiz, policy, please, lecture, week
Accounting terms: 0, Finance terms: 1
Sample documents: 14780.pdf, MGMT1560+Course+Outline.pdf, 2310a.pdf

Cluster 1 (likely Accounting - 'accounting' found in top terms):
Number of documents: 49
Top terms: accounting, finance, cost, statement, capital, management, business, risk, credit, cid
Sample documents: busn-0105.pdf, CourseraAcc.pdf, 405.pdf

Potential Outliers:
Number of outliers: 6
Outlier documents: CUBE.pdf, Mumbai.pdf, IITAcc.pdf, SharifFin2.pdf, DHFin.pdf
Error extracting text from your_file.pdf: [Errno 2] No such file or directory: 'your_file.pdf'

Classification Results for your_file.pdf:
Similarity Classification: Accounting (Confidence: 0.40, Outlier: T

In [ ]:
"""
Insights:

Cosine Similarity (including
1. Advantages: Efficient computation; in our work, the target accounting and finance syllabi documents are automatically selected by computer.
2. Disadvantages: Does not capture semantic meaning and needs vectorization; The results depend on the target syllabi, which might not be representative of the majority of syllabi.

Keywords Classification
1. Advantages: Simple and interpretable; We can just summarize necessary keywords based on what accounting or finance knowledge we think is important for applicants and define whether the student meets the requirements.
2. Disadvantages: For accounting and finance syllabi, many academic words (e.g. financial, taxation) appear in both types of syllabi, which may boost the score in an opposite way.

LDA
1. Advantages: It can automatically find some patterns and themes in documents, so we do not need training data. We can get some content information based on the topic of the text.
2. Disadvantages: The results could be sensitive to the number of topics we choose, and the words in each topic may include some useless information about content, which just appear frequently in the syllabi (e.g. student, class).

LSI
1. Advantages: It can handle mixed content well, given corresponding contribution or confidence scores.
2. Disadvantages: It will not be accurate when the documents are short and lack content.

Conclusion
Based on the pros and cons of the four methods, although it seems that LDA may give better results and be more accurate, our sample results showing the Keywords method gives the most accurate results in defining syllabi. One reason may be some unique finance keywords like “capm”, “beta”, “wacc” that boost our confidence level and increase the accuracy compared with other methods. However, it is also not completely reliable because we define the keywords by ourselves based on our knowledge of finance and accounting. Also, we found a difficulty in some special cases (e.g. BANKING-AND-FINANCE-PROGRAM-COURSE-CONTENT.pdf), which is supposed to be the outlier because it contains many different courses' information but classified as finance-related. Therefore, it is important to double-check the syllabi manually.

"""